# detect

In [ ]:
import os
import sys
import yaml
import torch
import numpy as np
import nibabel as nib
from tqdm import tqdm
from torchvision import transforms
from importlib import import_module

# Change working directory
os.chdir('/home/tanzl/code/githubdemo/THOR_DDPM')
from model_zoo.ddpm import DDPM
from PIL import Image

# Set input and output directories
# input_dir = sys.argv[1]
# output_dir = sys.argv[2]
input_dir = '/home/tanzl/data/mood/brainMRI/toy'
output_dir = '/home/tanzl/data/mood/brainMRI/output_toy'

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load configuration and model
config_path = '/home/tanzl/code/githubdemo/THOR_DDPM/projects/thor/configs/mood_brainMRI/mood_brainMRI.yaml'
pt_path = '/home/tanzl/code/githubdemo/THOR_DDPM/weights/thor/mood_brainMRI/trained_90epoch/best_model.pt'
noise_level = 350


# Load config
with open(config_path, 'r') as stream_file:
    config_file = yaml.load(stream_file, Loader=yaml.FullLoader)



# Load model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
global_model = torch.load(pt_path)

model = DDPM(**(config_file['model']['params']))
model.load_state_dict(global_model['model_weights'], strict=False)
model.eval()
model.to(device)

# Process NIfTI files
nifti_files = [f for f in os.listdir(input_dir)]

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
h, w = 256, 256

for nifti_file in nifti_files:
    nifti_path = os.path.join(input_dir, nifti_file)
    nifti = nib.load(nifti_path)
    data_array = nifti.get_fdata()
    affine_matrix = nifti.affine

    anomaly_maps = []

    for slice_idx in tqdm(range(data_array.shape[2])):  # Assuming the third dimension is the slice
        img = data_array[:, :, slice_idx]
        # print(np.max(img),np.min(img))
        if np.sum(img) < 200:
            anomaly_maps.append(np.zeros((h, w)))
            continue
        img = img*255
        img = Image.fromarray(img).convert('L').resize((128, 128))
        plt.imshow(img)
        plt.show()
        img = img.rotate(90, expand=True)
        plt.imshow(img)
        plt.show()
        img_tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)

        anomaly_map, _, _ = model.get_anomaly(img_tensor, noise_level=noise_level)
        print(np.max(anomaly_map),np.min(anomaly_map))
        anomaly_map = Image.fromarray(anomaly_map[0][0]*255).resize((h, w))
        plt.imshow(anomaly_map)
        plt.show()
        anomaly_map = anomaly_map.rotate(-90, expand=True)
        anomaly_map = (np.array(anomaly_map)/255)
        anomaly_map = np.clip(anomaly_map, 0, 1)
        print(np.max(anomaly_map),np.min(anomaly_map))
        plt.imshow(anomaly_map)
        plt.show()
        print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
        anomaly_maps.append(anomaly_map)

    # Stack all anomaly maps into a 3D array
    anomaly_map_3d = np.stack(anomaly_maps, axis=-1)

    
    # Save the anomaly map as a new NIfTI file
    output_nifti = nib.Nifti1Image(anomaly_map_3d, affine=affine_matrix)
    output_file = os.path.join(output_dir, f"anomaly_{nifti_file}")
    nib.save(output_nifti, output_file)

    break

In [ ]:
for i in range(len(anomaly_maps)):
    # if anomaly_maps[i].shape[2] == 1:
    anomaly_maps[i] = anomaly_maps[i].reshape(h,w)

In [ ]:
anomaly_map_3d = np.stack(anomaly_maps, axis=1)

In [ ]:
# Save the anomaly map as a new NIfTI file
output_nifti = nib.Nifti1Image(anomaly_map_3d, affine=affine_matrix)
output_file = os.path.join(output_dir, f"anomaly_{nifti_file}")
nib.save(output_nifti, output_file)

In [ ]:
anomaly_maps[128].shape

In [ ]:
output_file

# 判断是否存在异常

In [11]:
import numpy as np
import nibabel as nib
from scipy.ndimage import uniform_filter

def load_nifti(nifti_path):
    nifti = nib.load(nifti_path)
    data_array = nifti.get_fdata()
    return data_array

def detect_significant_anomaly(data_array, window_size=5, threshold_factor=2):
    # 计算局部平均
    local_mean = uniform_filter(data_array, size=window_size)

    # 计算全局平均和标准差
    global_mean = np.mean(data_array)
    global_std = np.std(data_array)

    # 定义显著程度的阈值
    threshold = global_mean + threshold_factor * global_std

    # 判断是否存在显著异常区域
    significant_anomaly = np.any(local_mean > threshold)

    return significant_anomaly

# 示例用法
window_size = 10
threshold_factor = 4

nifti_path = '/home/tanzl/data/mood/brainMRI/output_toy/anomaly_toy_0.nii.gz'
data_array = load_nifti(nifti_path)
exists_anomaly = detect_significant_anomaly(data_array=data_array,window_size=window_size, threshold_factor=threshold_factor)

print("存在显著异常区域" if exists_anomaly else "不存在显著异常区域")

不存在显著异常区域


In [ ]:
%load_ext autoreload
%autoreload 2

import yaml
import logging
import argparse
# import wandb
from datetime import datetime
import sys
sys.path.append('/home/tanzl/code/githubdemo/THOR_DDPM')
from dl_utils.config_utils import *
import warnings
from tqdm import tqdm

import copy

import os
print(os.getcwd())  # 检查当前工作目录
os.chdir('/home/tanzl/code/githubdemo/THOR_DDPM')
print(os.getcwd())  # 


from net_utils.simplex_noise import generate_noise, generate_simplex_noise

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np


def load_image_as_tensor(image_path):
    """
    读取图像并转换为形状为 (1, 1, 128, 128) 的 PyTorch 张量。
    """
    image = Image.open(image_path).convert('L')
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)
    return image_tensor

def display_images(img, rec):
    """
    将 img, rec 和 np.abs(img - rec) 以一行三列的方式展示。

    参数:
    img (numpy.ndarray): 第一个图像数组，形状为 (1, 128, 128)
    rec (numpy.ndarray): 第二个图像数组，形状为 (1, 128, 128)
    """
    # 计算绝对差值图像
    diff = np.abs(img - rec)

    # 去除第一个维度以便于显示
    img = img.squeeze()
    rec = rec.squeeze()
    diff = diff.squeeze()

    # 创建一个图形和子图
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # 显示 img
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title('Image')
    axes[0].axis('off')  # 关闭坐标轴

    # 显示 rec
    axes[1].imshow(rec, cmap='gray')
    axes[1].set_title('Reconstructed')
    axes[1].axis('off')  # 关闭坐标轴

    # 显示绝对差值图像
    axes[2].imshow(diff, cmap='gray')
    axes[2].set_title('Absolute Difference')
    axes[2].axis('off')  # 关闭坐标轴

    # 调整布局
    plt.tight_layout()
    plt.show()


In [ ]:
# 登录wandb
import wandb
wandb.init()


In [ ]:
# 加载config_file
config_path = '/home/tanzl/code/githubdemo/THOR_DDPM/projects/thor/configs/mood_brainMRI/mood_brainMRI.yaml'
# config_path = '/home/tanzl/code/githubdemo/THOR_DDPM/projects/thor/configs/brain/thor.yaml'
# config_path = '/home/tanzl/code/githubdemo/THOR_DDPM/projects/thor/configs/mood_brainMRI/mood_brainMRI_augmented.yaml'
stream_file = open(config_path, 'r')
config_file = yaml.load(stream_file, Loader=yaml.FullLoader)

In [ ]:
# 初始化DLConfigurator 
configurator_class = import_module(config_file['configurator']['module_name'],config_file['configurator']['class_name'])
configurator = configurator_class(config_file=config_file, log_wandb=True) # core.Configurator.DLConfigurator
exp_name = configurator.dl_config['experiment']['name'] # 'THOR'
method_name = configurator.dl_config['name'] # 'THOR [Gaussian] AD 350'

In [ ]:
# DLConfigurator.start_training(self, global_model: dict = dict())
# 加载data
data = configurator.load_data(configurator.dl_config['trainer']['data_loader'], train=True)
val_ds = data.val_dataloader()
train_ds = data.train_dataloader()

# 加载traner_class然后实例化trainer
trainer_class = import_module(configurator.dl_config['trainer']['module_name'], configurator.dl_config['trainer']['class_name']) # projects.thor.DDPMTrainer.PTrainer

trainer = trainer_class(training_params=configurator.dl_config['trainer']['params'], model=copy.deepcopy(configurator.model),
                                data=data, device=configurator.device, log_wandb=True)



In [ ]:
# # 一些额外的信息
# # DLConfigurator.load_data
# data_loader_config = configurator.dl_config['trainer']['data_loader']
# data_loader_module = import_module(data_loader_config['module_name'], data_loader_config['class_name'])



# configurator.model # DDPM
# configurator.dl_config['trainer']['params']
# '''
# {'input_size': (128, 128),
#  'checkpoint_path': './weights/thor/thor/',
#  'batch_size': 8,
#  'nr_epochs': 1500,
#  'patience': 1500,
#  'val_interval': 1,
#  'loss': {'module_name': 'optim.losses.ln_losses',
#   'class_name': 'L2',
#   'params': None},
#  'optimizer_params': {'lr': 0.0001}}
# '''
# trainer.transform # None

# configurator.dl_config['downstream_tasks']['detection']['checkpoint_path'] # './weights/thor/mood_brainMRI/'

# global_model.keys() # dict_keys(['model_weights', 'optimizer_weights', 'epoch'])

In [ ]:
# 加载模型
device = configurator.device
pt_path = '/home/tanzl/code/githubdemo/THOR_DDPM/weights/thor/mood_brainMRI/trained_90epoch/best_model.pt'
global_model = torch.load(pt_path)

# 执行下游任务

In [ ]:
configurator.start_evaluations(global_model['model_weights'], 350)

In [ ]:
for noise_level in tqdm(range(0,1000,50)):
    configurator.start_evaluations(global_model['model_weights'], noise_level)
# configurator.start_evaluations(global_model)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.show()

In [ ]:
configurator.dl_config['downstream_tasks']

In [ ]:
# 初始化下游任务类
# 定义下游任务的参数
dst_name = 'detection'
dst_config = configurator.dl_config['downstream_tasks'][dst_name]
downstream_class = import_module(dst_config['module_name'], dst_config['class_name'])
data = configurator.load_data(dst_config['data_loader'], train=False)
# 下游任务类
dst = downstream_class(dst_name, configurator.model, configurator.device, data, dst_config['checkpoint_path'])

In [ ]:
# dst.start_task(global_model=global_model,noise_level=noise_level)
global_model = global_model
noise_level = 350
dst.pathology_localization(global_model, 3, 71, True, noise_level=noise_level)



In [ ]:
import torch
import os
import copy
from PIL import Image
from torchvision import transforms

# def detect_anomalies(self, global_model, path, noise_level=350, batch_size=1):
#     """
#     执行异常检测并获取异常分数
#     :param global_model: dict, 模型参数
#     :param path: str, 图像或目录路径
#     :param noise_level: int, 噪声水平
#     :param batch_size: int, 批次大小
#     :return: x, x_rec, anomaly_score, anomaly_map
#     """
self = dst
path = '/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/toy/'
noise_level=350
batch_size=32

self.model.load_state_dict(global_model['model_weights'], strict=False)
self.model.eval()

# 判断路径是文件还是目录
if os.path.isfile(path):
    img = Image.open(path).convert('L').resize((128,128))
    img_tensor = transforms.ToTensor()(img).unsqueeze(0).to(self.device)

    x = img_tensor
    anomaly_map, anomaly_score, x_rec_dict = self.model.get_anomaly(copy.deepcopy(x), noise_level=noise_level)
    x_rec = x_rec_dict['x_rec'] if 'x_rec' in x_rec_dict.keys() else torch.zeros_like(x)
    x_rec = torch.clamp(x_rec, 0, 1)
    # return x, x_rec, anomaly_score, anomaly_map, x_rec_dict

elif os.path.isdir(path):
    img_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.png')]
    all_x, all_x_rec, all_anomaly_scores, all_anomaly_maps = [], [], [], []

    # 分批次进行预测
    for i in tqdm(range(0, len(img_files), batch_size)):
        batch_files = img_files[i:i+batch_size]
        imgs = [transforms.ToTensor()(Image.open(f).convert('L').resize((128,128))) for f in batch_files]
        img_tensor = torch.stack(imgs).to(self.device)

        print(img_tensor.shape)
        x = img_tensor
        anomaly_map, anomaly_score, x_rec_dict = self.model.get_anomaly(copy.deepcopy(x), noise_level=noise_level)
        x_rec = x_rec_dict['x_rec'] if 'x_rec' in x_rec_dict.keys() else torch.zeros_like(x)
        x_rec = torch.clamp(x_rec, 0, 1)

        all_x.append(x)
        all_x_rec.append(x_rec)
        all_anomaly_scores.append(anomaly_score)
        all_anomaly_maps.append(anomaly_map)

    # 拼接结果
    all_x = torch.cat(all_x, dim=0)
    all_x_rec = torch.cat(all_x_rec, dim=0)
    all_anomaly_scores = torch.tensor(np.concatenate(all_anomaly_scores))
    all_anomaly_maps = torch.tensor(np.concatenate(all_anomaly_maps))
    ano_scores = [torch.max(a).item() for a in all_anomaly_maps]

    # return all_x, all_x_rec, all_anomaly_scores, all_anomaly_maps

else:
    raise ValueError("提供的路径无效")

In [ ]:
all_anomaly_maps = torch.cat(all_anomaly_maps, dim=0)

In [ ]:
all_anomaly_maps = torch.tensor(np.concatenate(all_anomaly_maps))

In [ ]:
all_anomaly_scores[:1].shape

In [ ]:
ano_scores

In [ ]:
for idx, x in tqdm(enumerate(data['Atlas'])):
    img_tensor = x
    masks = x[1].to(device)
    masks[masks > 0] = 1
    if torch.sum(masks) > 0:
        x = img_tensor[0].to(device)
        anomaly_map, anomaly_score, x_rec_dict = self.model.get_anomaly(copy.deepcopy(x), noise_level=noise_level)
        x_rec = x_rec_dict['x_rec'] if 'x_rec' in x_rec_dict.keys() else torch.zeros_like(x)
        x_rec = torch.clamp(x_rec, 0, 1)

        break

In [ ]:
anomaly_score

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

x_np = x.cpu().detach().numpy().squeeze()
x_rec_np = x_rec.cpu().detach().numpy().squeeze()
anomaly_map_np = anomaly_map.squeeze()

# 创建子图
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 显示输入图像
axes[0].imshow(x_np, cmap='gray')
axes[0].set_title('Input Image')
axes[0].axis('off')

# 显示重建图像
axes[1].imshow(x_rec_np, cmap='gray')
axes[1].set_title('Reconstructed Image')
axes[1].axis('off')

# 显示异常图
im = axes[2].imshow(anomaly_map_np, cmap='plasma', vmin=0, vmax=1)
axes[2].set_title('Anomaly Map')
axes[2].axis('off')

# 添加颜色条
fig.colorbar(im, ax=axes[2])

# 显示图像
plt.tight_layout()
plt.show()



print(anomaly_score)

In [ ]:
torch.min(x)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

x_np = x.cpu().detach().numpy().squeeze()
x_rec_np = x_rec.cpu().detach().numpy().squeeze()
anomaly_map_np = anomaly_map.squeeze()

# 创建子图
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 显示输入图像
axes[0].imshow(x_np, cmap='gray')
axes[0].set_title('Input Image')
axes[0].axis('off')

# 显示重建图像
axes[1].imshow(x_rec_np, cmap='gray')
axes[1].set_title('Reconstructed Image')
axes[1].axis('off')

# 显示异常图
im = axes[2].imshow(anomaly_map_np, cmap='plasma', vmin=0, vmax=1)
axes[2].set_title('Anomaly Map')
axes[2].axis('off')

# 添加颜色条
fig.colorbar(im, ax=axes[2])

# 显示图像
plt.tight_layout()
plt.show()



print(anomaly_score)

# 模型的预测

In [ ]:
test_model = trainer.model # 也可以直接等于configurator.model
test_model.load_state_dict(global_model['model_weights'])
test_model.eval()
test_model.to(device)
print()

# 直接读取图片进行预测

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm




In [ ]:
img_root = '/home/tanzl/code/githubdemo/THOR_DDPM/'
img_name = './data/brainMRI/png/val/toy_2_slice_107.png'
img_path = img_root + img_name[1:]

task = 'mood_brainMRI'
metrics = {
    task + '_loss_rec': 0,
    task + '_loss_mse': 0,
    task + '_loss_pl': 0,
}
test_total = 0

with torch.no_grad():
    x = load_image_as_tensor(img_path)

    b, c, h, w = x.shape
    test_total += b
    x = x.to(device)
    x = (x * 2) - 1
    x_, _ = test_model.sample_from_image(x, noise_level=trainer.model.noise_level_recon)
    x = (x + 1) /2 
    x_ = (x_ + 1) / 2
    loss_rec = trainer.criterion_rec(x_, x)
    loss_mse = trainer.criterion_MSE(x_, x)
    loss_pl = trainer.criterion_PL(x_, x)

    metrics[task + '_loss_rec'] += loss_rec.item() * x.size(0)
    metrics[task + '_loss_mse'] += loss_mse.item() * x.size(0)
    metrics[task + '_loss_pl'] += loss_pl.item() * x.size(0)

    # 这里需要通过data加载出图片名称，找出对应的label图片，然后进行指标的计算。



# 单独拿出来一个进行可视化
rec = x_.detach().cpu()[0].numpy()
img = x.detach().cpu()[0].numpy()


for metric_key in metrics.keys():
    metric_name = task + '/' + str(metric_key)
    metric_score = metrics[metric_key] / test_total
    print({metric_name: metric_score})

In [ ]:
%matplotlib inline
display_images(img, rec)

# 调试dataset

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
data = train_ds.dataset.__getitem__(0)
augment = data[0].detach().cpu()[0].numpy()
origin = data[1].detach().cpu()[0].numpy()
# plt.imshow(augment)
# plt.show()
# plt.imshow(origin)
# plt.show()

In [ ]:
import cv2


img_path= '/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/train/00704_slice_075.png'
img = cv2.imread(img_path)
aug_path= '/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/train_augmented/00704_slice_075_augmented.png'
aug = cv2.imread(aug_path)
plt.imshow(img)
plt.show()
plt.imshow(aug)
plt.show()

# nii2png

In [ ]:
import nibabel as nib
import numpy as np
from PIL import Image
import os
from tqdm import tqdm

def nii_to_png(nii_path, output_dir):
    """
    将 NIfTI 文件中的每个切片保存为 PNG 图像。
    :param nii_path: 输入的 NIfTI 文件路径
    :param output_dir: 输出的 PNG 图像保存目录
    """
    # 加载 NIfTI 文件
    img = nib.load(nii_path)
    img_data = img.get_fdata()

    # 获取文件名（不包括扩展名）
    base_name = os.path.basename(nii_path).split('.')[0]

    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)
    print(img_data.shape)
    # 遍历每个切片
    for i in range(img_data.shape[2]):
        
        slice_data = img_data[:, :, i]
        
        # 归一化切片数据到 0-255
        slice_norm = 255 * (slice_data - np.min(slice_data)) / (np.max(slice_data) - np.min(slice_data))
        slice_uint8 = slice_norm.astype(np.uint8)
        
        # 创建 PIL 图像对象
        img_pil = Image.fromarray(slice_uint8)
        
        # 保存 PNG 图像
        output_path = os.path.join(output_dir, f"{base_name}_slice_{i:03d}.png")
        img_pil.save(output_path)

def batch_process_nii_files(nii_files, output_dir):
    """
    批量处理多个 NIfTI 文件，将它们转换为 PNG 图像。
    :param nii_files: NIfTI 文件路径的列表
    :param output_dir: 输出的 PNG 图像保存目录
    """
    for nii_path in tqdm(nii_files):
        nii_to_png(nii_path, output_dir)

# 示例用法
nii_root = '/home/tanzl/data/mood/brainMRI/output_toy/'
# nii_files = [os.path.join(nii_root,name) for name in os.listdir(nii_root)]
nii_files = ['/home/tanzl/data/mood/brainMRI/output_toy/anomaly_toy_2.nii.gz']
output_dir = '/home/tanzl/data/mood/brainMRI/output_toy/png'
batch_process_nii_files(nii_files, output_dir)

In [ ]:
img_data.shape

# 从一个文件夹到另一个文件夹转移图片

In [ ]:
import os
import shutil

def extract_specific_images(src_dir, dest_dir, start_num, end_num):
    """
    提取文件名尾部编号在指定范围内的图片文件到新目录。
    
    :param src_dir: 源图片目录路径
    :param dest_dir: 目标图片目录路径
    :param start_num: 编号范围的起始值（含）
    :param end_num: 编号范围的结束值（含）
    """
    # 创建目标目录（如果不存在）
    os.makedirs(dest_dir, exist_ok=True)
    
    # 遍历源目录中的所有文件
    for filename in os.listdir(src_dir):
        # 检查文件是否以.png结尾
        if filename.endswith('.png'):
            # 提取尾部编号
            base_name, ext = os.path.splitext(filename)
            tail_num_str = base_name.split('_')[-1]
            
            if tail_num_str.isdigit():
                tail_num = int(tail_num_str)
                
                # 检查编号是否在指定范围内
                if start_num <= tail_num <= end_num:
                    # 构建源文件路径和目标文件路径
                    src_path = os.path.join(src_dir, filename)
                    dest_path = os.path.join(dest_dir, filename)
                    
                    # 复制文件到目标目录
                    shutil.copy(src_path, dest_path)


# 示例用法
src_dir = '/home/tanzl/data/mood/ATLAS_val_png'
dest_dir = '/home/tanzl/data/mood/ATLAS_val_png_selected'
start_num = 90
end_num = 95

extract_specific_images(src_dir, dest_dir, start_num, end_num)

# 旋转一个文件夹中的图片并保存

In [ ]:
from PIL import Image
import os

def rotate_images_in_directory(directory, angle=90):
    """
    将指定目录中的所有 PNG 图片顺时针旋转指定角度。

    :param directory: 图片所在目录路径
    :param angle: 旋转角度，默认为90度（顺时针）
    """
    # 确保目录存在
    if not os.path.exists(directory):
        print(f"目录 {directory} 不存在")
        return

    # 遍历目录中的所有文件
    for filename in tqdm(os.listdir(directory)):
        # 检查文件是否以.png结尾
        if filename.endswith('.png'):
            # 构建文件路径
            file_path = os.path.join(directory, filename)
            
            # 打开图片
            with Image.open(file_path) as img:
                # 旋转图片
                rotated_img = img.rotate(-angle, expand=True)
                
                # 保存旋转后的图片，覆盖原文件
                rotated_img.save(file_path)

# 示例用法
directory = '/home/tanzl/code/githubdemo/THOR_DDPM/data/abtomMRI/png/toy'
rotate_images_in_directory(directory)

# 读取并保存csv文件

In [ ]:
import os
import csv

# 定义包含 PNG 文件的文件夹路径
folder_path = '/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/png/train_augmented'

# 获取文件夹中的所有 PNG 文件名
file_names = [f for f in os.listdir(folder_path) if f.endswith('.png')]

# 完整的文件路径
file_paths = [os.path.join(folder_path, f) for f in file_names]

# 定义 CSV 文件的输出路径
output_csv = '/home/tanzl/code/githubdemo/THOR_DDPM/data/brainMRI/splits/mood_brainMRI_train_augmented.csv'

# 将文件名写入 CSV 文件
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['filename'])  # 写入表头
    for file_path in file_paths:
        writer.writerow([file_path])

print(f'File names have been written to {output_csv}')